## 그룹연산(groupby)
- 특정 기준 적용하여 몇개의 그룹으로 분할하여 처리하는 것
- 데이터 집계, 변환, 필터링에 효율적
- [프로세스]
    - 분할(Split)
    - 적용(Apply)
    - 결합(Combine)
- 그룹단위로 평균, 최대, 최소 등과 같은 집계 메서드 사용
    

In [30]:
# 모듈 로딩
import pandas as pd
import numpy as np

df = pd.DataFrame(
    [
        ("bird", "Falconiformes", 389.0),
        ("bird", "Psittaciformes", 24.0),
        ("mammal", "Carnivora", 80.2),
        ("mammal", "Primates", np.nan),
        ("mammal", "Carnivora", 58),
    ],
    index=["falcon", "parrot", "lion", "monkey", "leopard"],
    columns=("class", "order", "max_speed"),
)
df

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0
lion,mammal,Carnivora,80.2
monkey,mammal,Primates,NaN
leopard,mammal,Carnivora,58.0


In [31]:
df.info(), df.columns, df.index

<class 'pandas.core.frame.DataFrame'>
Index: 5 entries, falcon to leopard
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   class      5 non-null      object 
 1   order      5 non-null      object 
 2   max_speed  4 non-null      float64
dtypes: float64(1), object(2)
memory usage: 160.0+ bytes


(None,
 Index(['class', 'order', 'max_speed'], dtype='object'),
 Index(['falcon', 'parrot', 'lion', 'monkey', 'leopard'], dtype='object'))

In [32]:
# 그룹 만들기 -> 객체변수명.groupby(컬럼명)
classgroup = df.groupby(by='class')
classgroup  # 메모리주소만 띄워진다.

In [33]:
# 반복이 가능한 객체 -> for ~ in 사용 가능 ( 
# (GroupBy.__iter__() Groupby iterator.)가 있으면 반복이 가능한 객체임을 알 수 있다. 

for group in classgroup:
    print(group)

('bird',        class           order  max_speed
falcon  bird   Falconiformes      389.0
parrot  bird  Psittaciformes       24.0)
('mammal',           class      order  max_speed
lion     mammal  Carnivora       80.2
monkey   mammal   Primates        NaN
leopard  mammal  Carnivora       58.0)


In [34]:
# groups를 이용하면 그룹화된 형태 표현 ( Dict {group name -> group labels}.)
# 그룹키:[요소 인덱스, ...]
classgroup.groups

{'bird': ['falcon', 'parrot'], 'mammal': ['lion', 'monkey', 'leopard']}

In [35]:
# groups를 이용하면 그룹화된 형태 표현 (Dict {group name -> group indices}.)
classgroup.indices

{'bird': array([0, 1], dtype=int64), 'mammal': array([2, 3, 4], dtype=int64)}

In [36]:
# 특정 그룹 키에 해당하는 데이터만 추출 get_group(키)
classgroup.get_group('bird')

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0


In [37]:
# 쪼개서 집어넣을 수도 있다.
for key, group in classgroup:
    print(f'key: {key}\n')
    print(f'group: {group}\n')  

key: bird

group:        class           order  max_speed
falcon  bird   Falconiformes      389.0
parrot  bird  Psittaciformes       24.0

key: mammal

group:           class      order  max_speed
lion     mammal  Carnivora       80.2
monkey   mammal   Primates        NaN
leopard  mammal  Carnivora       58.0



In [38]:
# 그룹전체에 대한 메서드 적용

classgroup.describe()
classgroup.describe(include='all')   # include= 'all'을 쓰면 숫자 아닌 값들도 나온다. 

order                                                           ...  \
       count unique            top freq mean  std  min  25%  50%  75%  ...   
class                                                                  ...   
bird       2      2  Falconiformes    1  NaN  NaN  NaN  NaN  NaN  NaN  ...   
mammal     3      2      Carnivora    2  NaN  NaN  NaN  NaN  NaN  NaN  ...   

       max_speed                                                           \
          unique top freq   mean         std   min     25%    50%     75%   
class                                                                       
bird         NaN NaN  NaN  206.5  258.093975  24.0  115.25  206.5  297.75   
mammal       NaN NaN  NaN   69.1   15.697771  58.0   63.55   69.1   74.65   

               
          max  
class          
bird    389.0  
mammal   80.2  

[2 rows x 22 columns]

In [39]:
classgroup.std()

,max_speed
class,
bird,258.093975
mammal,15.697771


In [40]:
# 특정 그룹에 대한 것만 처리 get_group(키) 사용
classgroup.get_group('bird')  # bird 그룹만 나온다. 

,class,order,max_speed
falcon,bird,Falconiformes,389.0
parrot,bird,Psittaciformes,24.0


In [41]:
birdgroup = classgroup.get_group('bird')
# 집계함수를 한꺼번에 적용 -> agg를 이용하면 함수를 합쳐서 처리 가능 ['함수명', '함수명', '함수명',...]
birdgroup.agg(['max', 'min', 'sum'])

,class,order,max_speed
max,bird,Psittaciformes,389.0
min,bird,Falconiformes,24.0
sum,birdbird,FalconiformesPsittaciformes,413.0


In [43]:
# 집계함수를 한꺼번에 적용 -> agg를 이용하면 함수를 합쳐서 처리 가능 {'컬럼명: '함수명', '컬럼명': '함수명', '컬럼명': '함수명',...}
birdgroup.agg({'order': 'max', 'max_speed': ['min', 'sum']})

,order,max_speed
max,Psittaciformes,NaN
min,NaN,24.0
sum,NaN,413.0


In [46]:
# 사용자 정의함수
def get_maxmin(x):
    return x.max()-x.min()

birdgroup.agg({'order': 'max', 'max_speed': [get_maxmin]})

,order,max_speed
max,Psittaciformes,NaN
get_maxmin,NaN,365.0


In [48]:
# 그룹에서 필터링하기 --> filter(T/F 조건)(불린조건도 사용가능)
birdgroup.filter(items=['order'])

birdgroup.filter(items=['max_speed'])

,max_speed
falcon,389.0
parrot,24.0


In [56]:
# max_speed 컬럼값이 300이상인 데이터만 출력  -> 과제
birdgroup[birdgroup['max_speed'] >= 300]

,class,order,max_speed
falcon,bird,Falconiformes,389.0


In [58]:
birdgroup.filter(items=['max_speed'])[birdgroup['max_speed'] >= 300]

,max_speed
falcon,389.0


In [60]:
df = pd.DataFrame(
    {
        "group": ["a", "a", "a", "a", "a", "b", "b", "b", "b", "b"],
        "value": [6, 7, 4, 5, 9, 3, 6, 10, 1, 2],
    }
)
df

,group,value
0,a,6
1,a,7
2,a,4
3,a,5
4,a,9
5,b,3
6,b,6
7,b,10
8,b,1
9,b,2


In [63]:
df.groupby('group')['value'].rank(method='average', ascending=False)

0    3.0
1    2.0
2    5.0
3    4.0
4    1.0
5    3.0
6    2.0
7    1.0
8    5.0
9    4.0
Name: value, dtype: float64

# 피벗(Pivot): 사용자 임의대로 데이터를 정렬하고 필터링 가능
- 많은 양 데이터에서 필요한 자료만 뽑아 새롭게 표 생성
